VARIÁVEIS PARA OS CÁLCULOS:

x = dias de uso – considerando apontamentos/inventário 

w = variação número de apontamentos comparado ao mês anterior  

k = programações 

q = recursos 

j =  RDOs 

In [1]:
from tqdm.notebook import tqdm
from django.db.models import Q
from django.db.models import Count
from datetime import datetime, timedelta
import calendar
import pandas as pd

In [3]:
road_companies = [x for x in Company.objects.all().values_list('name',flat=True).order_by('name')]
road_companies

['CCR - AutoBAn', 'CCR - Rio Sp', 'CCR - RodoAnel', 'CCR - SPVias']

In [11]:
today = datetime.now()-timedelta(days=30)
last_month = today - timedelta(days=5)
months = [last_month.month,today.month]
today,last_month,months

(datetime.datetime(2024, 4, 3, 14, 19, 40, 396126),
 datetime.datetime(2024, 3, 29, 14, 19, 40, 396126),
 [3, 4])

In [12]:
months

[3, 4]

In [13]:
last_days = [calendar.monthrange(today.year, mes)[1] for mes in months]
last_days

[31, 30]

# x = dias de uso

In [14]:
def calculo_dias_de_uso(porcentagem):
    if porcentagem < 25:
        return 0
    elif 25 <= porcentagem < 65:
        return 0.3
    elif 65 <= porcentagem < 80:
        return 0.65
    else:
        return 1

In [15]:
def x(months, company):
    porcen_dias = []

    for x, month in enumerate(months):
        total_days_in_month = last_days[x]
        qtd_dias = []

        for day in range(1, total_days_in_month + 1):
            qtd_dias.append(
                Reporting.objects.filter(
                    company__name=company,
                    updated_at__year=today.year,
                    updated_at__month=month,
                    updated_at__day=day,
                    created_at__year=today.year,
                    created_at__month=month,
                    created_at__day=day
                ).exclude(
                    created_by__username__in=['rlcs', 'neoviacs', 'arteriscs']
                ).only('uuid').count()
            )

        # Calcula a quantidade de dias com apontamento
        quantity = sum(1 for qtd in qtd_dias if qtd != 0)  # Se houve pelo menos 1 apontamento no dia resulta em 1
        # Calcula a porcentagem de dias de uso, em relação ao total de dias no mês
        porcentagem = quantity / total_days_in_month * 100
        porcentagem_dias_de_uso = calculo_dias_de_uso(porcentagem) #comentar pra ver a porcentagem 
        porcen_dias.append(porcentagem_dias_de_uso)

    return porcen_dias

In [16]:
resultados_por_empresa = {}

for company in tqdm(road_companies, desc='Empresas'):
    resultados_por_empresa[company] = x(months, company) # para cada empresa execcuta a função 'x'

# df = pd.DataFrame(resultados_por_empresa, index=months)
# df = df.transpose()
# df.index.name = 'Empresa'
# df.to_excel('nota_dias_de_uso.xlsx')

for company, resultados in resultados_por_empresa.items():
    for index, porcentagem_dias_de_uso in enumerate(resultados):
        print(f"Nota da unidade {company}, mês {months[index]}: {porcentagem_dias_de_uso}")

Empresas:   0%|          | 0/4 [00:00<?, ?it/s]

Nota da unidade CCR - AutoBAn, mês 3: 0.3
Nota da unidade CCR - AutoBAn, mês 4: 0.3
Nota da unidade CCR - Rio Sp, mês 3: 0.3
Nota da unidade CCR - Rio Sp, mês 4: 0.65
Nota da unidade CCR - RodoAnel, mês 3: 0.3
Nota da unidade CCR - RodoAnel, mês 4: 0
Nota da unidade CCR - SPVias, mês 3: 0.65
Nota da unidade CCR - SPVias, mês 4: 1


# w = variação número de apontamentos comparado ao mês anterior  

In [17]:
def calcular_variacao(atual, anterior):
    if anterior == 0 and atual > 0:
        return 1, "Aumento de 100%"
    if anterior == 0:
        return 0, "Sem apontamentos anteriores"
    variacao = (atual - anterior) / anterior * 100
    if variacao >= 0:
        return 1, "Positivo"
    elif -15 <= variacao < 0:
        return 0.7, "Regular"
    elif -25 <= variacao < -15:
        return 0.35, "Mediano"
    else:
        return 0, "Atenção"

In [18]:
# Reporting.objects.filter(Q(created_at__year=2024, created_at__month=3) &
#                         Q(updated_at__year=2024, updated_at__month=3),
#                         company__name='LCM - Gaspar').count()

In [19]:
w = []

for company in tqdm(road_companies):
    atual_count = Reporting.objects.filter(
        company__name=company,
        updated_at__year=today.year,
        updated_at__month=today.month,
        created_at__year=today.year,
        created_at__month=today.month
    ).exclude(
        created_by__username__in=['rlcs', 'neoviacs', 'arteriscs']
    ).only('uuid').count()
        
    anterior_count = Reporting.objects.filter(
        company__name=company,
        updated_at__year=last_month.year,
        updated_at__month=last_month.month,
        created_at__year=last_month.year,
        created_at__month=last_month.month
    ).exclude(
        created_by__username__in=['rlcs', 'neoviacs', 'arteriscs']
    ).only('uuid').count()

    atual = int(atual_count)
    anterior = int(anterior_count)

    # Calcular a variação
    variacao, categoria = calcular_variacao(atual, anterior)
    w.append((company, variacao))

    print(company, variacao)
    print()
    
# df_w = pd.DataFrame(w, columns=['Empresa', 'Variação'])
# df_w.to_excel('nota_variacao_apontamentos_comparado_mes_anterior.xlsx', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

CCR - AutoBAn 0

CCR - Rio Sp 0

CCR - RodoAnel 0

CCR - SPVias 0



# k = programações

In [20]:
def calcular_valor_programacoes(programacoes):
    num_programacoes = programacoes.count()
    
    if num_programacoes >= 4: # maior ou igual a 4 = 1
        return 1
    elif 2 <= num_programacoes <= 3: # entre 2 e 3 = 0.5
        return 0.5
    else:
        return 0 # menos de 2 prog = 0

In [21]:
def k(road_companies, today, last_month):
    result = {}

    for company in tqdm(road_companies):
        last = Job.history.filter(
            history_date__month=last_month.month,
            history_date__year=last_month.year,
            company__name=company,
            history_type='+'
        )

        valor_mes_anterior = calcular_valor_programacoes(last)

        print(f'{company}: Nota programações mês {last_month.month} = {valor_mes_anterior}')

        atual = Job.history.filter(
            history_date__month=today.month,
            history_date__year=today.year,
            company__name=company,
            history_type='+'
        )

        valor_atual = calcular_valor_programacoes(atual)

        print(f'{company}: Nota programações mês {today.month} = {valor_atual}')

        result[company] = (valor_atual, valor_mes_anterior)

#     df_result = pd.DataFrame(result.values(), index=result.keys(), columns=[last_month.month, today.month])
#     df_result.to_excel('nota_programacoes.xlsx')

    return result

k_result = k(road_companies, today, last_month)

  0%|          | 0/4 [00:00<?, ?it/s]

CCR - AutoBAn: Nota programações mês 3 = 1
CCR - AutoBAn: Nota programações mês 4 = 1
CCR - Rio Sp: Nota programações mês 3 = 1
CCR - Rio Sp: Nota programações mês 4 = 1
CCR - RodoAnel: Nota programações mês 3 = 1
CCR - RodoAnel: Nota programações mês 4 = 0.5
CCR - SPVias: Nota programações mês 3 = 1
CCR - SPVias: Nota programações mês 4 = 1


# q = recursos

In [22]:
def q(road_companies, today, last_month):
    result = {}

    for company in tqdm(road_companies):
        # Calcula a nota de recursos para o mês atual
        total_pu_today = ProcedureResource.objects.filter(
            reporting__company__name=company,
            creation_date__month=today.month,
            creation_date__year=today.year
        ).count()
        total_adm_today = DailyReportContractUsage.objects.filter(
            worker__company__name=company,
            created_at__month=today.month,
            created_at__year=today.year
        ).count()
        total_performance_today = FieldSurvey.objects.filter(
            contract__subcompany__company__name=company,
            created_at__month=today.month,
            created_at__year=today.year
        ).count()
        total_recursos_today = total_pu_today + total_adm_today + total_performance_today

        # Calcula a nota de recursos para o mês anterior
        total_pu_last_month = ProcedureResource.objects.filter(
            reporting__company__name=company,
            creation_date__month=last_month.month,
            creation_date__year=last_month.year
        ).count()
        total_adm_last_month = DailyReportContractUsage.objects.filter(
            worker__company__name=company,
            created_at__month=last_month.month,
            created_at__year=last_month.year
        ).count()
        total_performance_last_month = FieldSurvey.objects.filter(
            contract__subcompany__company__name=company,
            created_at__month=last_month.month,
            created_at__year=last_month.year
        ).count()
        total_recursos_last_month = total_pu_last_month + total_adm_last_month + total_performance_last_month

        # Determina a nota de recursos para cada mês
        nota_recursos_today = 1 if total_recursos_today > 0 else 0
        nota_recursos_last_month = 1 if total_recursos_last_month > 0 else 0

        # Adiciona as notas de recursos ao dicionário result
        result[company] = (nota_recursos_last_month, nota_recursos_today)

        print(f'{company}: Nota recursos mês {last_month.month}: {nota_recursos_last_month}')
        print(f'{company}: Nota recursos mês {today.month}: {nota_recursos_today}')

#     df = pd.DataFrame.from_dict(result, orient='index', columns=[last_month.month, today.month])

#     filename = 'nota_recursos.xlsx'
#     df.to_excel(filename)

    return result

q_result = q(road_companies, today, last_month)

  0%|          | 0/4 [00:00<?, ?it/s]

CCR - AutoBAn: Nota recursos mês 3: 0
CCR - AutoBAn: Nota recursos mês 4: 0
CCR - Rio Sp: Nota recursos mês 3: 1
CCR - Rio Sp: Nota recursos mês 4: 1
CCR - RodoAnel: Nota recursos mês 3: 0
CCR - RodoAnel: Nota recursos mês 4: 0
CCR - SPVias: Nota recursos mês 3: 0
CCR - SPVias: Nota recursos mês 4: 0


# j = RDOs

In [23]:
def calcular_valor_RDO(porcentagem): # nota referente a porcentagem de equipes com RDO criado
    if porcentagem >= 50:
        return 1
    elif 15 <= porcentagem < 50:
        return 0.5
    else:
        return 0 # menor de 15% = 0

In [24]:
def j(road_companies, today, last_month):
    result = {}

    for company in tqdm(road_companies):
        contador_atual = 0
        contador_anterior = 0
        firms = Firm.objects.filter(company__name=company)
        total_equipes = firms.count() # pega o total de equipes

        for firm in firms:
            if firm.firm_multiple_daily_report.filter(
                Q(date__month=today.month),
                Q(date__year=today.year)
            ).exists():
                contador_atual += 1

            if firm.firm_multiple_daily_report.filter(
                Q(date__month=last_month.month),
                Q(date__year=last_month.year)
            ).exists():
                contador_anterior += 1

        porcentagem_atual = (contador_atual / total_equipes) * 100 if total_equipes != 0 else 0
        valor_RDO_atual = calcular_valor_RDO(porcentagem_atual)

        porcentagem_anterior = (contador_anterior / total_equipes) * 100 if total_equipes != 0 else 0 
        valor_RDO_anterior = calcular_valor_RDO(porcentagem_anterior)

        result[company] = (valor_RDO_anterior, valor_RDO_atual)

        print(f'{company}: Nota RDO mês {last_month.month}: {valor_RDO_anterior}')
        print(f'{company}: Nota RDO mês {today.month}: {valor_RDO_atual}')

#     df = pd.DataFrame.from_dict(result, orient='index', columns=[last_month.month, today.month])

#     filename = 'nota_RDO.xlsx'
#     df.to_excel(filename)

    return result

j_result = j(road_companies, today, last_month)

  0%|          | 0/4 [00:00<?, ?it/s]

CCR - AutoBAn: Nota RDO mês 3: 0
CCR - AutoBAn: Nota RDO mês 4: 0
CCR - Rio Sp: Nota RDO mês 3: 0
CCR - Rio Sp: Nota RDO mês 4: 0
CCR - RodoAnel: Nota RDO mês 3: 0
CCR - RodoAnel: Nota RDO mês 4: 0
CCR - SPVias: Nota RDO mês 3: 0
CCR - SPVias: Nota RDO mês 4: 0


# CÁLCULOS PARA O HS 

# HS (básico) 

Exemplo:
    porcentagem_dias_de_uso = 0.3
    variacao_apontamentos = 1
Exemplo cálculo:
    hs_basico = 50 * 0.3 + 50 * 1
          = 15 + 50
          = 65

In [11]:
# def calcular_hs_basico(resultados_por_empresa, w, months):
#     hs_basico = []

#     for company, variacao in w:
#         hs_basico_mes_anterior = 0
#         for i, month in enumerate(months):
#             porcentagem_dias_de_uso = resultados_por_empresa[company][i]
#             variacao_apontamentos = variacao
            
#             hs_basico_value = 50 * porcentagem_dias_de_uso + 50 * variacao_apontamentos

#             if i > 0:
#                 hs_basico.append({'Empresa': company, 'HS Mês Anterior': hs_basico_mes_anterior, 'HS Mês Atual': hs_basico_value})
#             hs_basico_mes_anterior = hs_basico_value 

#             company_str = str(company).ljust(30)
#             month_str = str(month).ljust(10)
#             print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos} | Nota: {hs_basico_value}")

#     return hs_basico

# resultados_hs_basico = calcular_hs_basico(resultados_por_empresa, w, months)

# df_result = pd.DataFrame(resultados_hs_basico)
# df_result.to_excel('hs_basico.xlsx', index=False, columns=['Empresa', 'HS Mês Anterior', 'HS Mês Atual'])

Empresa: LCM - Criciúma                 | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Nota: 50
Empresa: LCM - Criciúma                 | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Nota: 65.0
Empresa: LCM - Gaspar                   | Mês: 2          | Porcentagem de dias de uso: 0.65 | Variação de apontamentos: 0 | Nota: 32.5
Empresa: LCM - Gaspar                   | Mês: 3          | Porcentagem de dias de uso: 1 | Variação de apontamentos: 0 | Nota: 50
Empresa: LCM - Ituporanga               | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Nota: 50
Empresa: LCM - Ituporanga               | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Nota: 65.0


# HS (intermediário) 

Exemplo:
    porcentagem_dias_de_uso = 0.3
    variacao_apontamentos = 1
    valor_programacoes = 0
Exemplo de cálculo:
    hs_intermediario = 34 * 0.3 + 34 * 1 + 32 * 0
                     = 10.2 + 34 + 0
                     = 44.2

In [23]:
# def calcular_hs_intermediario(resultados_por_empresa, w, k_result, months):
#     hs_intermediario = {}

#     for company, variacao in w:
#         hs_intermediario[company] = []
#         hs_mes_anterior = 0

#         for i, month in enumerate(months):
#             porcentagem_dias_de_uso = resultados_por_empresa[company][i]
#             variacao_apontamentos = variacao
#             valor_programacoes = k_result[company][i]
            
#             hs_intermediario_value = 34 * porcentagem_dias_de_uso + 34 * variacao_apontamentos + 32 * valor_programacoes

#             company_str = str(company).ljust(30)
#             month_str = str(month).ljust(10)
#             print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos} | Valor de programações: {valor_programacoes} | Nota Intermediária: {hs_intermediario_value}")


#             if not hs_intermediario[company]:
#                 hs_intermediario[company].append((0, hs_intermediario_value))
#             else:
#                 hs_intermediario[company][-1] = (hs_intermediario[company][-1][1], hs_intermediario_value)

#     data = []
#     for company, values in hs_intermediario.items():
#         for i, (previous_hs_value, hs_value) in enumerate(values):
#             data.append([company, previous_hs_value, hs_value])

#     df_result = pd.DataFrame(data, columns=['Empresa', 'HS Mês Anterior', 'HS Mês Atual'])

#     filename = 'hs_intermediario.xlsx'
#     df_result.to_excel(filename, index=False)

#     return hs_intermediario

# resultados_hs_intermediario = calcular_hs_intermediario(resultados_por_empresa, w, k_result, months)

Empresa: Afirma Engenharia Viária       | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 0 | Valor de programações: 0 | Nota Intermediária: 0
Empresa: Afirma Engenharia Viária       | Mês: 3          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 0 | Valor de programações: 0 | Nota Intermediária: 0
Empresa: ELO Ecopistas                  | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 0.7 | Valor de programações: 0.5 | Nota Intermediária: 50.0
Empresa: ELO Ecopistas                  | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 0.7 | Valor de programações: 0.5 | Nota Intermediária: 50.0
Empresa: ELO Ecovias dos Imigrantes     | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 0 | Valor de programações: 1 | Nota Intermediária: 32
Empresa: ELO Ecovias dos Imigrantes     | Mês: 3          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 0 | Valo

# HS (avançado) 

Exemplo:
    Porcentagem de uso: 0.3
    Variação de apontamentos: 1 (Positivo)
    Nota programações: 0
    Nota Recursos = 0
    Nota RDO: 0
Exemplo de cálculo:
    hs_avancado_value = 20 * (0.3 + 1 + 0 + 0 + 0)
                      = 20 * 1.3
                      = 26

In [26]:
def calcular_hs_avancado(resultados_por_empresa, w, k_result, q_result, j_result, months):
    hs_avancado = {}
    hs_export_avancado = {}

    for company, variacao in w:
        hs_avancado[company] = []
        hs_export_avancado[company] = []

        for i, month in enumerate(months):
            porcentagem_dias_de_uso = resultados_por_empresa[company][i]
            variacao_apontamentos = variacao
            valor_programacoes = k_result[company][i]
            nota_recursos_atual = q_result[company][i]
            nota_rdo = j_result[company][i] 

            company_str = str(company).ljust(30)
            month_str = str(month).ljust(10)
            
            print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos} | Valor de programações: {valor_programacoes} | Nota de recursos: {nota_recursos_atual} | Nota RDO: {nota_rdo}")

            hs_avancado_value = 20 * porcentagem_dias_de_uso + 20 * variacao_apontamentos + 20 * valor_programacoes + 20 * nota_recursos_atual + 20 * nota_rdo
            print(f"HS Avançado para a empresa {company} no mês {month}: {hs_avancado_value}\n")

            hs_avancado[company].append(hs_avancado_value)

    return hs_avancado

hs_avancado = calcular_hs_avancado(resultados_por_empresa, w, k_result, q_result, j_result, months)

Empresa: CCR - AutoBAn                  | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 0 | Valor de programações: 1 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa CCR - AutoBAn no mês 3: 26.0

Empresa: CCR - AutoBAn                  | Mês: 4          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 0 | Valor de programações: 1 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa CCR - AutoBAn no mês 4: 26.0

Empresa: CCR - Rio Sp                   | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 0 | Valor de programações: 1 | Nota de recursos: 1 | Nota RDO: 0
HS Avançado para a empresa CCR - Rio Sp no mês 3: 46.0

Empresa: CCR - Rio Sp                   | Mês: 4          | Porcentagem de dias de uso: 0.65 | Variação de apontamentos: 0 | Valor de programações: 1 | Nota de recursos: 1 | Nota RDO: 0
HS Avançado para a empresa CCR - Rio Sp no mês 4: 53.0

Empresa: CCR - RodoAnel          

In [34]:
df=pd.DataFrame.from_dict(data=hs_export_avancado,orient='index',columns=months)

In [35]:
df

,3,4
CCR - AutoBAn,26.0,26.0
CCR - Rio Sp,46.0,53.0
CCR - RodoAnel,16.0,20.0
CCR - SPVias,33.0,40.0


In [37]:
df.to_excel('HS-CCR-'+'-'.join(str(a) for a in months)+'.xlsx')